In [ ]:
import preprocess as pp

In [ ]:
pp.make_dirs(pp.DIRECTORY_LIST)


In [ ]:
band_list = pp.yaml_to_band_index(pp.params)

In [ ]:
pp.load_and_stack_bands(pp.SOURCE_IMGS, band_list, pp.STACKED)

In [ ]:
product_list = os.listdir(pp.SOURCE_IMGS)
filtered_product_list = [band for band in product_list if band[-5] in band_list]
filtered_product_list = sorted(filtered_product_list)
filtered_product_paths = [os.path.join(pp.SOURCE_IMGS, fname) for fname in filtered_product_list]

In [ ]:
filtered_product_paths

In [ ]:
import os

In [ ]:
product_list = os.listdir(pp.SOURCE_IMGS)
# below works because only products that are bands have a int in the 4th to last position
filtered_product_list = [band for band in product_list if band[-5] in band_list]

In [ ]:
filtered_product_list[0][:-10] + ".tif"

In [ ]:
product_list[6][-5]

In [ ]:
product_list[6][-5]

In [ ]:
' in band_list